In [ ]:
import pandas as pd
from sqlalchemy.orm import sessionmaker#,relationship, backref
from sqlalchemy import create_engine,select
#from Schema import *
from utils_modeling import *

In [ ]:
def get_metric_ts(metrc,idx,instrument):
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    sql = select([Metrics]).where(and_(Metrics.instrument_id == instrument,
        Metrics.Metric == metrc,Metrics.equity_fundamental_id == idx)).select_from(Metrics)
    result=s.execute(sql)
    df_ = pd.DataFrame(result.fetchall()).drop_duplicates()
    df_.columns = result.keys()
    s.close()
    return df_

def get_metrics(instrument):
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    sql = select([Metrics]).where(Metrics.instrument_id == instrument).select_from(Metrics)
    result=s.execute(sql)
    df_ = pd.DataFrame(result.fetchall()).drop_duplicates()
    df_.columns = result.keys()
    s.close()
    return df_

In [ ]:
df1=get_metric_ts("Open",2,379)
df2=get_metric_ts("Market Cap",1,379)
df3=get_metric_ts("Day's Range",2,379)
df4=get_metric_ts("Avg. Volume",2,379)
df5=get_metric_ts("1y Target Est",1,379)
df6=get_metric_ts("Enterprise Value 3",2,379)

df7=get_metric_ts("Trailing P/E",2,379)
df8=get_metric_ts("Forward P/E 1",2,379)
df9=get_metric_ts("No. of Analysts",8,379)
df10=get_metric_ts("Avg. Estimate",8,379)
df11=get_metric_ts("Low Estimate",8,379)
df12=get_metric_ts("High Estimate",8,379)
df13=get_metric_ts("Year Ago EPS",8,379)

In [ ]:
df4

In [ ]:
result=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13])
result=result[(result.Period!="Next Qtr. (Sep 2018)")&(result.Period!="Current Year (2018)")&(result.Period!="Next Year (2019)")]
result=result.sort_values(["refreshed_at","Metric"])

instrument=get_data(Instrument)[["id","name"]]
equity=get_data(Equity_fundamentals)

result=pd.merge(result,instrument,left_on="instrument_id",right_on="id")
result=pd.merge(result,equity,left_on="equity_fundamental_id",right_on="id")
result=result.sort_values(["refreshed_at","Metric"])
result=result[["name","Category","Table","Metric","Period","Value","refreshed_at"]]
result.columns=["Ticker","Category","Section","Metric","Period","Value","Date"]
#result.set_index("Date")
#result["Date2"]=pd.to_datetime(result['Date'])#, format='%Y-%m-%d %H:%M:%S')
result=result.sort_values(["Date"])

In [ ]:
#result['Date']

In [ ]:
#result["DATE2"]= ts.dt.date
#result["DATE2"]=pd.to_datetime(result['Date'], format="%Y-%m-%d")
result["DATE2"]=result['Date'].dt.date.astype(str)

In [ ]:
result.head()

In [ ]:
result[result.DATE2=="2018-05-20"]

In [ ]:
result.DATE2.drop_duplicates().shape

In [ ]:
#result.groupby(['DATE2']).count()
result.groupby(['Metric']).count()

In [ ]:
result.to_csv("SBUX.csv")

In [ ]:
all_data=get_metrics(379)

In [ ]:
all_data.head()
instrument=get_data(Instrument)[["id","name"]]
equity=get_data(Equity_fundamentals)

result=pd.merge(all_data,instrument,left_on="instrument_id",right_on="id")
result=pd.merge(result,equity,left_on="equity_fundamental_id",right_on="id")
result=result.sort_values(["refreshed_at","Metric"])
result=result[["name","Category","Table","Metric","Period","Value","refreshed_at"]]
result.columns=["Ticker","Category","Section","Metric","Period","Value","Date"]
#result.set_index("Date")
#result["Date2"]=pd.to_datetime(result['Date'])#, format='%Y-%m-%d %H:%M:%S')
result=result.sort_values(["Date"])
result.head()

In [ ]:
result.to_csv("SBUX.csv")

#  TEST DOWNLOAD OPTIONS INFO FROM YAHOO

In [ ]:
import sys
sys.path.append('/home/ubuntu/quantum/Infrastructure')
#from download_historical_equity_fundamental import *
from Schema import *
from mvo_utils import *
sys.path.append('home/ubuntu/quantum/Modeling')

In [ ]:
def get_summary(name):
    try:
        name1=name.replace('WIKI/' ,'')
        summaryf=get_data_sf(pd.read_html('https://finance.yahoo.com/quote/'+name1+'/options?p='+name1))[1]
        summaryf.columns=["Metric","Value"]
        summaryf["Category"]="Summary";summaryf["Table"]="Summary"
        summaryf["Period"]=str(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d'))
        summaryf["refreshed_at"]=datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        summaryf["instrument_id"]=get_id_instrument(name)
        summaryf["data_vendor_id"]=get_id_data_vendor("Yahoo_Finance")
        summaryf["equity_fundamental_id"]=summaryf[["Category","Table"]].apply(lambda x :add_up_equityf(x["Category"],x["Table"]), axis=1)
        summaryf[["instrument_id","equity_fundamental_id","data_vendor_id","refreshed_at","Period","Metric","Value"]].to_csv(Path_stats+"summary_"+name1+".csv")
        return True
    except Exception as err:
        print("Error: {0}".format(err))
        print("No se existe summary para "+name)
        return err
        pass
    
# https://finance.yahoo.com/quote/SBUX/options?p=SBUX
def get_data_sf(data_request):
    count=0
    while True:
        try:
            count=count+1
            summary = data_request ; break
            if count==limit:
                break;
        except Exception as err:
            print("Error: {0}, waiting to try again in {1}".format(err, sleep_time))
            sleep(sleep_time)
            if count==limit:
                break;
    return summary

In [ ]:
name="SBUX"

name1=name.replace('WIKI/' ,'')
summaryf=get_data_sf(pd.read_html("https://www.marketwatch.com/investing/stock/SBUX/options?countrycode=US&showAll=True"))[0]

In [ ]:
summaryf.columns=["var0","var1","var2","var3","var4","var5","var6","var7","var8","var9","var10","var11","var12","var13","var14"]
summaryf=summaryf[summaryf.var0.notnull()]
summaryf.index=range(summaryf.shape[0])

In [ ]:
#summaryf['var0']
summaryf.shape[0]

In [ ]:
idx_subtable=summaryf[summaryf['var0'].str.contains('Expires')].index.tolist()
idx_subtable.append(summaryf.shape[0])
idx_subtable

In [ ]:
lista=[[idx_subtable[i],idx_subtable[i+1]]for i in range(len(idx_subtable)-1)]
lista

In [ ]:
df_final=pd.DataFrame()
for range_ in lista:
    idx_0=range_[0]
    idx_1=range_[1]
    aux=summaryf.loc[idx_0]["var0"].replace('Expires ', '').replace(',', '')
    datetime_object = datetime.strptime(aux,'%B %d %Y').date()
    df_aux=summaryf.loc[idx_0+1:idx_1]#[summaryf.var1.notnull()]
    df_aux=df_aux[df_aux.var1.notnull()]
    df1=df_aux[["var1","var2","var3","var4","var5","var6","var7"]]
    df1["type"]='Call'
    df1.columns=["last","change","volume","bid","ask","open_interest","strike","type"]

    df2=df_aux[["var9","var10","var11","var12","var13","var14","var7"]]
    df2.columns=["last","change","volume","bid","ask","open_interest","strike"]
    df2["type"]="Put"

    df_final_aux=df1.append(df2)
    df_final_aux["expiration"]=datetime_object
    df_final_aux["date_"]=datetime.today().strftime("%Y-%m-%d")
    df_final=df_final.append(df_final_aux)

In [ ]:
df_final

In [ ]:
#summaryf.loc[4:54].head()

In [ ]:
aux=summaryf.loc[3]["var0"].replace('Expires ', '').replace(',', '')
from datetime import datetime
print(aux)
datetime_object = datetime.strptime(aux,'%B %d %Y').date()
datetime_object

In [ ]:
#df_aux=summaryf.loc[5:54][summaryf.var1.notnull()]
df_aux=summaryf.loc[5:54]#[summaryf.var1.notnull()]
df_aux=df_aux[df_aux.var1.notnull()]
df1=df_aux[["var1","var2","var3","var4","var5","var6","var7"]]
df1["type"]='Call'
df1.columns=["last","change","volume","bid","ask","open_interest","strike","type"]

df2=df_aux[["var9","var10","var11","var12","var13","var14","var7"]]
df2.columns=["last","change","volume","bid","ask","open_interest","strike"]
df2["type"]="Put"

df_final=df1.append(df2)
df_final["expiration"]=datetime_object
df_final["date_"]=datetime.today().strftime("%Y-%m-%d")
df_final

In [ ]:
call.head()

In [2]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup as BS
import datetime
import pandas as pd

#url ="https://finance.yahoo.com/quote/SBUX/options?p=SBUX" #"https://www.barchart.com/stocks/quotes/SBUX/options"#
def get_list_dates(name):
    url = "https://query2.finance.yahoo.com/v7/finance/options/"+name
    usock = urlopen(url)
    data = usock.read()
    usock.close()
    soup = BS(data,"lxml")

    python_dict = json.loads(soup.getText())
    list_dates=python_dict['optionChain']['result'][0]['expirationDates']
    return list_dates

name="SBUX"

def get_options(name):
    call=pd.DataFrame()
    put=pd.DataFrame()
    name1=name.replace('WIKI/' ,'')
    list_dates=get_list_dates(name)
    #print(list_dates)
    for j in list_dates:
        #print(j)
        try:
            calls=pd.read_html("https://finance.yahoo.com/quote/"+name+"/options?p="+name+"&date="+str(j))[0]
            calls["expiration"]=datetime.datetime.fromtimestamp(int(j)).strftime('%Y-%m-%d')
            call=call.append(calls)
        except Exception as err:
            print("Error: {0}".format(err))
        try:
            puts=pd.read_html("https://finance.yahoo.com/quote/"+name+"/options?p="+name+"&date="+str(j))[1]
            puts["expiration"]=datetime.datetime.fromtimestamp(int(j)).strftime('%Y-%m-%d')
            put=put.append(puts)
        except Exception as err:
            print("Error: {0}".format(err))
    call["type"]="call"
    put["type"]="put"
    final=call.append(put)
    final["ticker"]=name
    return final

In [3]:
options_sbux=get_options(name)

Error: No tables found
Error: No tables found


In [4]:
options_sbux.head()

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,expiration,type,ticker
0,SBUX180706C00041500,2018-07-06 6:51PM EDT,41.5,7.40,5.65,8.05,0.00,-,1,1,307.42%,2018-07-06,call,SBUX
1,SBUX180706C00042500,2018-07-06 1:25PM EDT,42.5,6.64,5.50,6.95,0.53,+8.67%,3,3,258.98%,2018-07-06,call,SBUX
2,SBUX180706C00044000,2018-07-05 10:14AM EDT,44.0,4.50,4.65,5.25,0.00,-,1,14,178.52%,2018-07-06,call,SBUX
3,SBUX180706C00044500,2018-07-06 6:51PM EDT,44.5,4.50,4.25,4.75,0.00,-,1,6,92.19%,2018-07-06,call,SBUX
4,SBUX180706C00045000,2018-06-29 11:47PM EDT,45.0,3.75,1.70,6.45,0.00,-,1,0,113.28%,2018-07-06,call,SBUX


In [ ]:
options_sbux.to_csv("SBUX_options.csv")

# Analisis options SBUX

In [6]:
options_sbux

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,expiration,type,ticker
0,SBUX180706C00041500,2018-07-06 6:51PM EDT,41.5,7.40,5.65,8.05,0.00,-,1,1,307.42%,2018-07-06,call,SBUX
1,SBUX180706C00042500,2018-07-06 1:25PM EDT,42.5,6.64,5.50,6.95,0.53,+8.67%,3,3,258.98%,2018-07-06,call,SBUX
2,SBUX180706C00044000,2018-07-05 10:14AM EDT,44.0,4.50,4.65,5.25,0.00,-,1,14,178.52%,2018-07-06,call,SBUX
3,SBUX180706C00044500,2018-07-06 6:51PM EDT,44.5,4.50,4.25,4.75,0.00,-,1,6,92.19%,2018-07-06,call,SBUX
4,SBUX180706C00045000,2018-06-29 11:47PM EDT,45.0,3.75,1.70,6.45,0.00,-,1,0,113.28%,2018-07-06,call,SBUX
5,SBUX180706C00045500,2018-06-27 2:56PM EDT,45.5,4.55,2.99,5.85,0.00,-,2,2,230.08%,2018-07-06,call,SBUX
6,SBUX180706C00047000,2018-07-06 10:03AM EDT,47.0,1.70,1.81,2.27,-0.04,-2.30%,13,279,58.59%,2018-07-06,call,SBUX
7,SBUX180706C00047500,2018-07-06 3:50PM EDT,47.5,1.43,1.45,1.59,0.25,+21.19%,31,67,56.25%,2018-07-06,call,SBUX
8,SBUX180706C00048000,2018-07-06 3:52PM EDT,48.0,0.95,0.92,1.10,0.17,+21.79%,163,505,44.53%,2018-07-06,call,SBUX
9,SBUX180706C00048500,2018-07-06 3:51PM EDT,48.5,0.46,0.41,0.59,0.15,+48.39%,158,310,28.32%,2018-07-06,call,SBUX
